In [278]:
## Library Import

# Basic
import re
import warnings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

# Preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Scaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer

# AutoML optuna library
import optuna
import optuna.integration.lightgbm as lgb

# Linear Model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

# Tree Model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Metric
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

In [279]:
# data load
df = pd.read_csv('original_full_data.csv', index_col = 0)
mmdf = pd.read_csv('bloomberg_map_index_20200130.csv', index_col = 0)

In [280]:
# From the date the target data is present ~
df = df.loc[9132 :, :]

In [281]:
df.reset_index(inplace = True, drop = True)

In [282]:
# Class generates derived variables & shifting target variable

class Builder:

    def __init__(self, input_df, map_df, target_day):
        self.df = input_df
        self.mdf = map_df
        self.lag_days = [1, 5, 20, 60, 120, 260] # lagging day variation [1day 1week 1month 3month 6month 1year]
        self.target_day = target_day # ex) 1, 260


    # shifting function

    def targeter(self, target_column):
        self.df[target_column] = self.df[target_column].shift(-1*int(self.target_day))

        return self.df
    
    # generating derived columns function : return ex) 20060107/20060101

    def returner(self, target_column):      

        col_name =  []
        # creating new column name
        for i in self.lag_days:
            col_name.append(target_column + str(i))
            
        # appending lagged columns
        for i, j in enumerate(self.lag_days):
            self.df[locals()['col_name'][i]] = self.df[target_column] / self.df[target_column].shift(periods = j, axis = 0)

        return self.df 

    # generating derived function : minus ex) 20060107 - 20060101
    
    def differ(self, target_column):

        col_name = []
        for i in self.lag_days:
            col_name.append(target_column + str(i))
        for i, j in enumerate(self.lag_days):
            self.df[locals()['col_name'][i]] = self.df[target_column] - self.df[target_column].shift(periods = j, axis = 0)
        return self.df

    # generating derived function : volume * close  -> convert to return 

    def producter(self, target_column):

        col_name = (str(target_column).split('_'))[0]       
        self.df[locals()['col_name'] + '_product'] = self.df[col_name + '_volume'] * self.df[col_name + '_close'] # j volume과 j close를 곱해서 

        return self.df
    
    # executing functions
    
    def execution(self):
        for i, j in enumerate(self.mdf['ticker']): # mdf에서 ticker 확인
            if j in self.df.columns:
                if self.mdf.loc[i, :][1] == 'product':  # key값이 product인 경우
                    self.producter(j)
                    col = str(j).split('_')[0]
                    self.df[col + '_product'] = self.df[col + '_product'] / self.df[col + '_product'].shift(1)
                    
                #elif self.mdf.loc[i, :][1] == 'minus': # key값이 minus인 경우
                #    self.differ(j)

                #elif self.mdf.loc[i, :][1] == 'change': # key값이 change인 경우
                #    self.returner(j)


                elif self.mdf.loc[i, :][1] == 'target': # key값이 target인 경우 (현재 KRXsemiconductor_change)
                    self.targeter(j)

                else:
                    pass
            else:
                pass
                            
        return self.df


    # side : creating auto correlation graph function

    def acf_cal(self, target):
        data = pd.DataFrame(self.df[target])
        data = data.interpolate(method = 'cubic', limit_area = 'inside').fillna(method = 'ffill').fillna(method = 'bfill')
        acf = sm.tsa.acf(data)      
        plt.stem(acf)
        plt.show()

In [283]:
newdf = df.copy()

In [284]:
len(newdf.columns)

417

In [285]:
# date preprocessing

newdf['date'] = pd.to_datetime(newdf['date'])
newdf.set_index('date', inplace = True)

In [286]:
# fillna cubic -> ffill -> bfill

newdf = newdf.interpolate(method = 'cubic', limit_area = 'inside')
newdf = newdf.fillna(method = 'ffill')
newdf = newdf.fillna(method = 'bfill')

In [287]:
# check na value

newdf.isna().sum().sum()

0

In [288]:
# class load

cla = Builder(newdf, mmdf, 120)

In [289]:
# execution function
# 새로운 데이터프레임 생성 
cla.execution()

,GDPCQOQIndex,GDPCYOYIndex,GDPCTOT%Index,GPDITOC%Index,GPGSTOC%Index,RGCDCIPIIndex,GDPCUR$Index,GDPCURYIndex,GDPPIQQIndex,GDPCPCECIndex,CPIYOYIndex,CPIXYOYIndex,PCEDEFYIndex,PCECYOYIndex,CPICHNGIndex,CPUPXCHGIndex,FDIDFDMOIndex,FDIDSGMOIndex,FDIUFDYOIndex,FDIUSGYOIndex,USURTOTIndex,USUDMAERIndex,INJCJCIndex,INJCSPIndex,NFPTCHIndex,...,AMD_product,APPLE_product,AppliedMaterials_product,Aspeed_product,Dell_product,Facebook_product,Google_product,HPE_product,Intel_product,LamResearch_product,MediaTek_product,Micron_product,Microsoft_product,Nuvoton_product,Nvidia_product,Philadelphia_product,QCOM_product,SamsungElectronics_product,ShinEtsuChemical_product,Siltronic_product,SKhynix_product,Sumco_product,TokyoElectron_product,TSMC_product,UMC_product
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2006-01-02,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.700000,8.400000,326.0,2561.0,278.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-03,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.700000,8.400000,326.0,2561.0,278.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.862009,1.000000,1.000000,1.048766,1.000000,1.000000,1.000000,1.000000
2006-01-04,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.700000,8.400000,326.0,2561.0,278.000000,...,0.682030,0.771197,1.026287,1.000000,1.000000,1.000000,1.223552,1.000000,0.829714,0.967572,1.000000,1.380558,0.743634,1.000000,1.109161,0.967923,1.097724,2.307115,1.000000,1.000000,1.549763,1.000000,1.000000,0.788606,0.581332
2006-01-05,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.700000,8.400000,326.0,2561.0,278.000000,...,1.264163,0.718678,1.094089,1.000000,1.000000,1.000000,0.710449,1.000000,1.204864,0.824310,1.000000,0.901202,0.836536,1.000000,1.294707,1.168471,0.642743,0.878133,1.000000,1.000000,0.913503,1.000000,4.149537,0.835657,2.780952
2006-01-06,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.700000,8.400000,326.0,2561.0,278.000000,...,0.966656,1.606885,1.514631,1.000000,1.000000,1.000000,1.685084,1.000000,0.833842,1.282477,1.000000,0.947752,2.059981,1.000000,0.980556,1.047620,2.572150,0.504080,1.000000,1.000000,0.596882,1.000000,1.117321,1.553101,1.545230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-27,1.9,2.0,2.9,-1.5,2.0,69.0,21525.82,3.7,1.7,2.2,1.7,2.4,1.3,1.66611,0.0,0.1,-0.3,-0.3,1.4,2.0,3.543428,6.908116,218.0,1690.0,128.639710,...,0.900501,1.010875,1.764097,0.820641,0.808665,1.410060,1.415943,0.954658,0.776571,2.092578,0.829932,1.022473,1.347411,1.486079,0.898487,0.954025,1.223596,-7.016632,1.443049,0.882592,0.724875,0.591650,1.185079,0.728242,8.957344
2019-10-28,1.9,2.0,2.9,-1.5,2.0,69.0,21525.82,3.7,1.7,2.2,1.7,2.4,1.3,1.66611,0.0,0.1,-0.3,-0.3,1.4,2.0,3.556001,6.928135,218.0,1690.0,128.219228,...,1.213115,1.395378,1.417193,0.968297,1.516745,1.109701,1.114263,1.105058,0.581300,1.697246,3.008876,0.982582,1.005550,0.918295,0.709275,0.988696,0.869484,3.268112,0.911087,0.984864,0.598464,0.436636,1.103782,1.596400,-4.887075
2019-10-29,1.9,2.0,2.9,-1.5,2.0,69.0,21525.82,3.7,1.7,2.2,1.7,2.4,1.3,1.66611,0.0,0.1,-0.3,-0.3,1.4,2.0,3.569604,6.950086,218.0,1690.0,127.968071,...,1.506408,1.445075,0.717119,1.250066,1.680148,1.011607,0.793725,1.167880,0.557896,0.832619,1.914615,0.789692,0.580119,0.417745,0.626504,1.023202,0.493114,1.174228,0.410159,1.089446,0.832789,1.231150,1.131356,1.864625,2.523049


In [290]:
# fill na for shifted data

newdf = cla.df.fillna(method = 'ffill').fillna(method = 'bfill')

In [291]:
# check na

newdf.isna().sum().sum()

0

In [292]:
# features

len(newdf.columns)

442

In [293]:
newdf.drop('SamsungElectronics_product', inplace = True, axis = 1)

In [294]:
newdf.to_csv('googleautomltest_120_200226.csv')

# Scaling

In [178]:
newdf.reset_index(inplace = True)

In [179]:
newdf.head()

,date,GDPCQOQIndex,GDPCYOYIndex,GDPCTOT%Index,GPDITOC%Index,GPGSTOC%Index,RGCDCIPIIndex,GDPCUR$Index,GDPCURYIndex,GDPPIQQIndex,GDPCPCECIndex,CPIYOYIndex,CPIXYOYIndex,PCEDEFYIndex,PCECYOYIndex,CPICHNGIndex,CPUPXCHGIndex,FDIDFDMOIndex,FDIDSGMOIndex,FDIUFDYOIndex,FDIUSGYOIndex,USURTOTIndex,USUDMAERIndex,INJCJCIndex,INJCSPIndex,...,AMD_product,APPLE_product,AppliedMaterials_product,Aspeed_product,Dell_product,Facebook_product,Google_product,HPE_product,Intel_product,LamResearch_product,MediaTek_product,Micron_product,Microsoft_product,Nuvoton_product,Nvidia_product,Philadelphia_product,QCOM_product,SamsungElectronics_product,ShinEtsuChemical_product,Siltronic_product,SKhynix_product,Sumco_product,TokyoElectron_product,TSMC_product,UMC_product
0,2006-01-02,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.7,8.4,326.0,2561.0,...,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,0.862009,1.0,1.0,1.048766,1.0,1.000000,1.000000,1.000000
1,2006-01-03,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.7,8.4,326.0,2561.0,...,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,0.862009,1.0,1.0,1.048766,1.0,1.000000,1.000000,1.000000
2,2006-01-04,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.7,8.4,326.0,2561.0,...,0.682030,0.771197,1.026287,1.0,1.0,1.0,1.223552,1.0,0.829714,0.967572,1.0,1.380558,0.743634,1.0,1.109161,0.967923,1.097724,2.307115,1.0,1.0,1.549763,1.0,1.000000,0.788606,0.581332
3,2006-01-05,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.7,8.4,326.0,2561.0,...,1.264163,0.718678,1.094089,1.0,1.0,1.0,0.710449,1.0,1.204864,0.824310,1.0,0.901202,0.836536,1.0,1.294707,1.168471,0.642743,0.878133,1.0,1.0,0.913503,1.0,4.149537,0.835657,2.780952
4,2006-01-06,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.7,8.4,326.0,2561.0,...,0.966656,1.606885,1.514631,1.0,1.0,1.0,1.685084,1.0,0.833842,1.282477,1.0,0.947752,2.059981,1.0,0.980556,1.047620,2.572150,0.504080,1.0,1.0,0.596882,1.0,1.117321,1.553101,1.545230


In [180]:
date = newdf.date

In [181]:
newdf.drop('date', axis = True, inplace = True)

In [182]:
# extract target
y_target = newdf['KRXsemiconductor_change']

In [215]:
# target drop
newdf.drop(['KRXsemiconductor_change'], axis = 1, inplace = True)


In [216]:
# 삼성전자가 이상하게 안됨 점검 필요 자꾸 infinity가 뜨므로 일단 제거
newdf.drop('SamsungElectronics_product', inplace = True, axis = 1)

In [185]:
# 이 지점에서 각자 잡은 feature를 넣어주자


newdf = newdf[['KOSPI200_change', 'Dell_volume', 'KOIMPTIIndex120', 'USWHTOTIndex120', 'MTSLRL$Index1', 'LEINWCNIndex5', 'SKLIMORDIndex260', 'KOIPMCIndex5', 'CONCCONFIndex60', 'OUTFGAFIndex20', 'KOHHDIndex20', 'QCOM_change', 'OEKRN022Index1', 'KOIMTOTIndex60', 'KOBPFINIndex260', 'EMPRGBCIIndex120', 'MPMIJPMAIndex20', 'ShinEtsuChemical_change', 'COMFCOMFIndex120', 'ShinEtsuChemical_close', 'KODIBALIndex260', 'MPMIUSMAIndex5', 'KOECSEMQIndex', 'KOBPFINIndex60', 'MPMIUSSAIndex120', 'FRNTTOTLIndex120', 'KOFDITIndex', 'INJCJCIndex20', 'KOWDRIndex5', 'MPMIJPMAIndex5', 'USDollarIndex_change', 'MTSLRL$Index5', 'KWCDCCurncy1', 'NAPMNEWOIndex5', 'KOPSIYIndex', 'Nvidia_product', 'KOBPFINIndex1', 'Intel_product', 'SKLIMORDIndex120', 'Facebook_product', 'KODSDISCIndex', 'LamResearch_change', 'UMC_change', 'LEIMNOIndex60', 'KOBSNMCIndex120', 'KORSTIndex260', 'EMDINP1MIndex1', 'AppliedMaterials_volume', 'KOHSTRIndex', 'Philadelphia_product', 'TSMC_volume', 'KOHHLIndex1', 'USWHTOTIndex1', 'KOWDRIndex120', 'LEIAVGWIndex5', 'USTBIMPIndex5', 'KOWDRIndex60', 'FRNTTNETIndex60', 'Micron_product', 'TSMC_change', 'TokyoElectron_change', 'g2', 'Micron_volume', 'KOTRBALIndex5', 'ShinEtsuChemical_volume', 'SKLISVCIIndex60', 'SKLIWNRSIndex1', 'NAPMPMIIndex5', 'APPLE_product', 'KOBPCAIndex260', 'SKLISVCIIndex120', 'KOIPOPSMIndex', 'MediaTek_change', 'KOBPCAIndex', 'KOBONTLIndex5', 'AMD_product', 'KOEXTOTIndex20', 'SKhynix_change', 'NAPMNMIIndex1', 'SKLILIIndex1', 'KOFDITIndex1', 'FRNTTNETIndex260', 'MAPMINDXIndex1', 'KOFDITIndex5', 'AppliedMaterials_product', 'KOEXTOTIndex260', 'KOSPI200_volume', 'IPIndex120', 'KOHHDIndex1', 'Microsoft_product', 'KOMSM2YIndex', 'KOCGCGSMIndex', 'SKhynix_product', 'UMC_product', 'USTBTOTIndex20', 'KOGFBALIndex1', 'OEKRN022Index5', 'KOHCTTLIndex', 'LEIWKIJIndex60', 'EMPRGBCIIndex20', 'SKLICONEIndex120', 'Amazon_product', 'FRNTTOTLIndex1', 'SKLINBARIndex5', 'EUR_KRW_volume', 'JPY_KRW_volume', 'USDollarIndex_volume', 'USD_KRW_volume', 'PIDSPINXIndex120', 'EOKOS002Index', 'KOEXPTIIndex20', 'KOEXPTIYIndex', 'LEIAVGWIndex1', 'KOULMGFIndex120', 'PITLCHNGIndex', 'KOBSNMCIndex5', 'KOBSNMCIndex20', 'ShinEtsuChemical_product', 'Google_product', 'TokyoElectron_product', 'GVSK10YRIndex', 'SKBSICIndex60', 'UMC_volume', 'GDPPIQQIndex', 'INJCSPIndex5', 'EMPRGBCIIndex5', 'SKLILIIndex60', 'SKCITTLIndex20', 'Aspeed_close', 'SKLIMORDIndex60', 'AWHTOTLIndex60', 'CONCCONFIndex20', 'Amazon_volume', 'TokyoElectron_volume']]

KeyError: "['LEIAVGWIndex5', 'KOIMPTIIndex120', 'KOHHLIndex1', 'KOEXPTIIndex20', 'KOEXTOTIndex260', 'SKLIMORDIndex260', 'KOHHDIndex1', 'KOBPFINIndex60', 'INJCSPIndex5', 'SKLINBARIndex5', 'KODIBALIndex260', 'INJCJCIndex20', 'FRNTTOTLIndex120', 'KORSTIndex260', 'EMPRGBCIIndex5', 'SKLILIIndex60', 'FRNTTNETIndex260', 'USWHTOTIndex1', 'SKLIMORDIndex120', 'SKBSICIndex60', 'MPMIUSSAIndex120', 'CONCCONFIndex60', 'COMFCOMFIndex120', 'MTSLRL$Index1', 'OEKRN022Index5', 'MPMIJPMAIndex20', 'NAPMNMIIndex1', 'KWCDCCurncy1', 'KOBSNMCIndex5', 'KOFDITIndex5', 'KOULMGFIndex120', 'SKCITTLIndex20', 'KOWDRIndex5', 'KOBPFINIndex1', 'KOHHDIndex20', 'LEIMNOIndex60', 'CONCCONFIndex20', 'SKLISVCIIndex60', 'SKLIWNRSIndex1', 'KOIMTOTIndex60', 'KOWDRIndex120', 'KOWDRIndex60', 'MTSLRL$Index5', 'SKLISVCIIndex120', 'KOBPFINIndex260', 'SKLICONEIndex120', 'OUTFGAFIndex20', 'NAPMNEWOIndex5', 'KOIPMCIndex5', 'KOBPCAIndex260', 'NAPMPMIIndex5', 'FRNTTOTLIndex1', 'PIDSPINXIndex120', 'EMDINP1MIndex1', 'KOBSNMCIndex20', 'AWHTO

In [130]:
len(newdf.columns)

110

,GDPCQOQIndex,GDPCYOYIndex,GDPCTOT%Index,GPDITOC%Index,GPGSTOC%Index,RGCDCIPIIndex,GDPCUR$Index,GDPCURYIndex,GDPPIQQIndex,GDPCPCECIndex,CPIYOYIndex,CPIXYOYIndex,PCEDEFYIndex,PCECYOYIndex,CPICHNGIndex,CPUPXCHGIndex,FDIDFDMOIndex,FDIDSGMOIndex,FDIUFDYOIndex,FDIUSGYOIndex,USURTOTIndex,USUDMAERIndex,INJCJCIndex,INJCSPIndex,NFPTCHIndex,...,Amazon_product,AMD_product,APPLE_product,AppliedMaterials_product,Aspeed_product,Dell_product,Facebook_product,Google_product,HPE_product,Intel_product,LamResearch_product,MediaTek_product,Micron_product,Microsoft_product,Nuvoton_product,Nvidia_product,Philadelphia_product,QCOM_product,ShinEtsuChemical_product,Siltronic_product,SKhynix_product,Sumco_product,TokyoElectron_product,TSMC_product,UMC_product
0,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.7,8.4,326.0,2561.0,278.0,...,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.048766,1.0,1.000000,1.000000,1.000000
1,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.7,8.4,326.0,2561.0,278.0,...,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.048766,1.0,1.000000,1.000000,1.000000
2,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.7,8.4,326.0,2561.0,278.0,...,0.990868,0.682030,0.771197,1.026287,1.0,1.0,1.0,1.223552,1.0,0.829714,0.967572,1.0,1.380558,0.743634,1.0,1.109161,0.967923,1.097724,1.0,1.0,1.549763,1.0,1.000000,0.788606,0.581332
3,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.7,8.4,326.0,2561.0,278.0,...,0.735048,1.264163,0.718678,1.094089,1.0,1.0,1.0,0.710449,1.0,1.204864,0.824310,1.0,0.901202,0.836536,1.0,1.294707,1.168471,0.642743,1.0,1.0,0.913503,1.0,4.149537,0.835657,2.780952
4,5.4,3.4,4.5,6.1,5.2,102.7,13603.93,6.6,2.8,2.3,4.0,2.1,3.2,2.06622,0.6,0.2,0.1,0.3,2.6,2.2,4.7,8.4,326.0,2561.0,278.0,...,1.139744,0.966656,1.606885,1.514631,1.0,1.0,1.0,1.685084,1.0,0.833842,1.282477,1.0,0.947752,2.059981,1.0,0.980556,1.047620,2.572150,1.0,1.0,0.596882,1.0,1.117321,1.553101,1.545230


In [131]:
# scaling 하기전에 columns 뽑기 
col_name = newdf.columns

In [132]:
# scaling과 동시에 변환
#newdf = PowerTransformer().fit_transform(newdf)
newdf = StandardScaler().fit_transform(newdf)
#newdf = newdf.apply(lambda x : np.log1p(x))
#newdf = MinMaxScaler().fit_transfrom(newdf)

In [133]:
# scaling하면 colname사라지므로 다시 생성
newdf = pd.DataFrame(newdf, columns = col_name)

In [134]:
len(newdf.columns)

110

In [135]:
newdf.isna().sum().sum()

0

In [136]:
# extract X_data
X_data = newdf.copy()

In [137]:
# random split과 not random split
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size = 0.3, random_state = 156)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size = 0.2, shuffle = False)

# Linear Feature Selection
## 안돌려도 무방

In [138]:
'''
## Feature Selection Code
# Forward stepwise feature selection
# linear model 기준으로 유효 feature 뽑기. 돌리지 않아도 되는 코드

np.random.seed(0)

feature_name = list(X_train.columns)
feature_selected = []
score = []

for i in range(X_train.shape[1]):
    score_temp_list = np.zeros(len(feature_name))
    for j in range(len(feature_name)):
        temp = feature_selected.copy() 
        temp.append(feature_name[j])
        x_temp = X_train[temp]
        lr_temp = LinearRegression() 
        lr_temp.fit(x_temp, y_train)
        predict_temp = lr_temp.predict(X_test[temp])
        r2_temp = r2_score(y_test, predict_temp)
        score_temp = r2_temp
        score_temp_list[j] = score_temp 
        
    temp = feature_name[score_temp_list.argmax()]
    feature_selected.append(temp)
    feature_name.remove(temp)
    score.append(score_temp_list.max())
    print("%02d Selected: "%i, feature_selected)
    print("%02d Score : "%i, np.round(10000*np.array(score))/10000)
        
# finally selected features
feature_selected_final = feature_selected[:(np.array(score).argmax() + 1)]

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-138-dcc4a0f6e932>, line 33)

# Metric Class

In [139]:
# metric 구성

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score

class CustomMetric():

    def __init__(self, pred, real):
            self.pred = pred
            self.real = real
            self.thr25 = np.percentile(self.real, 25)
            self.thr75 = np.percentile(self.real, 75)
            self.df = pd.DataFrame({'pred' : self.pred, 'real' : self.real})
        
    def issame(self, data1, data2):
        if data1 > 0 and data2 > 0 : 
            return 'T'
        elif data1 > 0 and data2 < 0 :
            return 'F'
        elif data1 < 0 and data2 > 0 :
            return 'F'
        elif data1 < 0 and data2 < 0 :
            return 'T'
        elif data1 == 0 or data2 == 0 :
            return 0
        else :
            return 'notcal'

    def getouter(self, data1, data2): #quantile_25 = np.percentile(values, 25)

        if data1 > 0 and data2 >= self.thr75 :
            return 'T'
        elif data1 < 0 and data2 <= self.thr25 :
            return 'T'
        elif data1 >= 0 and data2 <= self.thr25 :
            return 'F'
        elif data1 <= 0 and data2 >= self.thr75: 
            return 'F'
        else:
            return 'notcal'
        
    def makedf(self):
        self.df['TF'] = self.df.apply(lambda x : self.issame(x['pred'], x['real']), axis = 1)
        self.df['thrTF'] = self.df.apply(lambda x : self.getouter(x['pred'], x['real']), axis = 1)
        
        return self.df
        
    def execution(self):
        mdf = pd.DataFrame()
        mdf['CORR'] = [self.df['real'].corr(self.df['pred'], method = 'pearson')]
        mdf['R2'] = ["{0:0.4f}".format(r2_score(self.df['real'], self.df['pred']))]
        mdf['MAE'] = ["{0:0.4f}".format(mean_absolute_error(self.df['pred'], self.df['real']))]
        mdf['RMSE'] = ["{0:0.4f}".format(np.sqrt(mean_squared_error(self.df['pred'], self.df['real'])))]
        mdf['ACR'] = [sum(self.df['TF'] == 'T')/len(self.df['TF'])]
        
        mdf['threshACR'] = [sum(self.df['thrTF'] == 'T') / sum(self.df['thrTF'] != 'notcal')]
        
        return mdf     

# Linear Model

In [140]:
# linear model select
lr_reg = LinearRegression()
#ridge = Ridge(alpha = 100)
#lasso = Lasso(alpha = 100)

In [141]:
lr_reg.fit(X_train, y_train)
y_pred = lr_reg.predict(X_test)

# Tree Model

In [98]:
dt_reg = DecisionTreeRegressor(random_state = 0, max_depth = 100)
rf_reg = RandomForestRegressor(random_state = 0, n_estimators = 1000)
gb_reg = GradientBoostingRegressor(random_state = 0, n_estimators = 1000)
xgb_reg = XGBRegressor(n_estimators = 1000)
lgb_reg = LGBMRegressor(n_estimators = 3000)

In [ ]:
lgb_reg.fit(X_train, y_train)

In [44]:
y_pred = lgb_reg.predict(X_test)

### Metric function 실행 

In [142]:
metr = CustomMetric(y_pred, y_test)
metr.makedf()
metr.execution()

,CORR,R2,MAE,RMSE,ACR,threshACR
0,0.315524,0.0987,1.4147,1.8702,0.570722,0.609467


# Feature Importance Visualization

Linear Model은 회귀계수로 나오므로 아래와 같이 시각화

In [ ]:
coef_imp = pd.DataFrame(sorted(zip(lr_reg.coef_, X_train.columns)), columns = ['Value', 'Feature'])
coef_imp.sort_values(by = 'Value', ascending = False)
plt.figure(figsize = (10, 30))
sns.barplot(x = 'Value', y = 'Feature', data = coef_imp.sort_values(by = 'Value', ascending = False))
plt.title('Linear Regressor Features')
plt.tight_layout()
plt.show()

Tree Model은 feature importance가 따로 뽑히므로 아래와 같이 시각화

In [ ]:
feat_imp = pd.DataFrame(sorted(zip(lgb_reg.feature_importances_, X_train.columns)), columns = ['Value', 'Feature'])
feat_imp.sort_values(by = 'Value', ascending = False)
plt.figure(figsize = (10, 30))
sns.barplot(x = 'Value', y = 'Feature', data = feat_imp.sort_values(by = 'Value', ascending = False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.savefig('lgbm_importances_1.png')
plt.show()

# 번외 : Optuna AutoML을 통한 LightGBM Hyperparameter tuning 예시

In [183]:
import sklearn.ensemble
import sklearn.model_selection
from sklearn.metrics import r2_score
import optuna
import optuna.integration.lightgbm as lgb

In [184]:
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_test, label=y_test)

params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
}

best_params, tuning_history = dict(), list()

model = lgb.train(params,
                  dtrain,
                  valid_sets=[dtrain, dval],
                  best_params=best_params,
                  tuning_history=tuning_history,
                  verbose_eval=100,
                  early_stopping_rounds=100,
                  )

prediction = np.rint(model.predict(X_test, num_iteration=model.best_iteration))
r2_score = r2_score(y_test, prediction)

print('Number of finished trials: {}'.format(len(tuning_history)))
print('Best params:', best_params)
print('  r2_score = {}'.format(r2_score))
print('  Params: ')
for key, value in best_params.items():
    print('    {}: {}'.format(key, value))

tune_feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds


tune_feature_fraction, val_score: 3.743039:  14%|#4        | 1/7 [00:00<00:03,  1.96it/s]

[100]	training's l2: 1.14427	valid_1's l2: 4.37173
Early stopping, best iteration is:
[2]	training's l2: 4.35093	valid_1's l2: 3.74304
Training until validation scores don't improve for 100 rounds


tune_feature_fraction, val_score: 3.743039:  29%|##8       | 2/7 [00:01<00:02,  1.98it/s]

...

[100]	training's l2: 2.23093	valid_1's l2: 4.53729
Early stopping, best iteration is:
[2]	training's l2: 4.44043	valid_1's l2: 3.7674
Training until validation scores don't improve for 100 rounds


tune_min_child_samples, val_score: 3.673303:  80%|########  | 4/5 [00:01<00:00,  3.90it/s]

[100]	training's l2: 2.43349	valid_1's l2: 4.29193
Early stopping, best iteration is:
[1]	training's l2: 4.52346	valid_1's l2: 3.77838
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 2.72726	valid_1's l2: 4.30335


tune_min_child_samples, val_score: 3.673303: 100%|##########| 5/5 [00:01<00:00,  3.87it/s]

Early stopping, best iteration is:
[2]	training's l2: 4.48874	valid_1's l2: 3.74655
Number of finished trials: 65
Best params: {'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 15, 'feature_fraction': 0.4, 'bagging_fraction': 0.7225765831000759, 'bagging_freq': 1, 'min_child_samples': 5}
  r2_score = -0.0458964390586587
  Params: 
    lambda_l1: 0.0
    lambda_l2: 0.0
    num_leaves: 15
    feature_fraction: 0.4
    bagging_fraction: 0.7225765831000759
    bagging_freq: 1
    min_child_samples: 5


### Tuning된 값 입력하여 모델 선언

In [57]:
lgb_reg = LGBMRegressor(lambda_l1 = 0.6885870703417802, lambda_l2 = 1.1741445421730667, num_leaves = 33, feature_fraction = 0.8999999999999999, bagging_fraction = 1.0, bagging_freq = 0, min_child_samples = 20)

In [58]:
lgb_reg.fit(X_train, y_train)

LGBMRegressor(bagging_fraction=0.4999, bagging_freq=6, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.616,
              importance_type='split', lambda_l1=0, lambda_l2=0,
              learning_rate=0.1, max_depth=-1, min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
              n_jobs=-1, num_leaves=50, objective=None, random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [59]:
y_pred = lgb_reg.predict(X_test)